# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
# Function to filter rows where start/stop range overlap with the specified range
def filter_overlapping_ranges(frame_ranges, start=0, stop=np.inf):
    # Check for overlap
    overlaps = np.logical_or(np.logical_and(frame_ranges[:,0] < stop, frame_ranges[:,0] >= start),
                             np.logical_and(frame_ranges[:,1] <= stop, frame_ranges[:,1] > start))
    return frame_ranges[overlaps]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [9]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_comp_discriminate"

In [10]:
TRODES_METADATA_DF = pd.read_pickle("./proc/{}_00_trodes_metadata.pkl".format(OUTPUT_PREFIX))

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [12]:
TRODES_METADATA_DF = TRODES_METADATA_DF.drop(columns=["current_subject", "video_timestamps", "recording", "session_path"], errors="ignore")
TRODES_METADATA_DF = TRODES_METADATA_DF.drop_duplicates(subset=["video_name"])

In [13]:
exploded_TRODES_METADATA_DF = TRODES_METADATA_DF.explode(column = ["tone_frames", "tone_timestamps"])

In [14]:
exploded_TRODES_METADATA_DF

,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240413_151204_comp_discriminate_subj_3-1_and...,"[1, 66]","[[1, 66], [23242, 23292], [23292, 23299], [237...","[[1, 66], [261, 272], [563, 578], [732, 738], ...",20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,"[0, 65243]","[[0, 65243], [22819129, 22858929], [22859529, ...","[[0, 65243], [260845, 271845], [563249, 578649..."
0,20240413_151204_comp_discriminate_subj_3-1_and...,"[1264, 1464]","[[1, 66], [23242, 23292], [23292, 23299], [237...","[[1, 66], [261, 272], [563, 578], [732, 738], ...",20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,"[1265655, 1465660]","[[0, 65243], [22819129, 22858929], [22859529, ...","[[0, 65243], [260845, 271845], [563249, 578649..."
0,20240413_151204_comp_discriminate_subj_3-1_and...,"[3260, 3459]","[[1, 66], [23242, 23292], [23292, 23299], [237...","[[1, 66], [261, 272], [563, 578], [732, 738], ...",20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,"[3265681, 3465685]","[[0, 65243], [22819129, 22858929], [22859529, ...","[[0, 65243], [260845, 271845], [563249, 578649..."
0,20240413_151204_comp_discriminate_subj_3-1_and...,"[4458, 4657]","[[1, 66], [23242, 23292], [23292, 23299], [237...","[[1, 66], [261, 272], [563, 578], [732, 738], ...",20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,"[4465698, 4665701]","[[0, 65243], [22819129, 22858929], [22859529, ...","[[0, 65243], [260845, 271845], [563249, 578649..."
0,20240413_151204_comp_discriminate_subj_3-1_and...,"[5455, 5656]","[[1, 66], [23242, 23292], [23292, 23299], [237...","[[1, 66], [261, 272], [563, 578], [732, 738], ...",20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,"[5465711, 5665716]","[[0, 65243], [22819129, 22858929], [22859529, ...","[[0, 65243], [260845, 271845], [563249, 578649..."
...,...,...,...,...,...,...,...,...,...,...,...
25,20240417_150153_comp_discriminate_subj_3-3_and...,"[64047, 64246]","[[1, 151], [22661, 22676], [22676, 22692], [23...","[[1, 151], [233, 236], [237, 250], [250, 280],...",20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,"[64052556, 64252558]","[[0, 151362], [22702244, 22717241], [22718241,...","[[0, 151362], [233765, 236563], [237965, 24956..."
25,20240417_150153_comp_discriminate_subj_3-3_and...,"[66242, 66443]","[[1, 151], [22661, 22676], [22676, 22692], [23...","[[1, 151], [233, 236], [237, 250], [250, 280],...",20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,"[66252586, 66452588]","[[0, 151362], [22702244, 22717241], [22718241,...","[[0, 151362], [233765, 236563], [237965, 24956..."
25,20240417_150153_comp_discriminate_subj_3-3_and...,"[67340, 67540]","[[1, 151], [22661, 22676], [22676, 22692], [23...","[[1, 151], [233, 236], [237, 250], [250, 280],...",20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,"[67352600, 67552600]","[[0, 151362], [22702244, 22717241], [22718241,...","[[0, 151362], [233765, 236563], [237965, 24956..."
25,20240417_150153_comp_discriminate_subj_3-3_and...,"[68938, 69137]","[[1, 151], [22661, 22676], [22676, 22692], [23...","[[1, 151], [233, 236], [237, 250], [250, 280],...",20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,"[68952620, 69152620]","[[0, 151362], [22702244, 22717241], [22718241,...","[[0, 151362], [233765, 236563], [237965, 24956..."


In [15]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: np.array(x))


box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [16]:
columns_to_update = ["box_1_port_entry_frames", "box_2_port_entry_frames", "box_1_port_entry_timestamps", "box_2_port_entry_timestamps"]
tone_columns = ["tone_frames", "tone_frames", "tone_timestamps", "tone_timestamps"]

for col, tone_col in zip(columns_to_update, tone_columns):
    exploded_TRODES_METADATA_DF[col] = exploded_TRODES_METADATA_DF.apply(lambda x: filter_overlapping_ranges(x[col], x[tone_col][0], x[tone_col][1]), axis=1)

In [17]:
exploded_TRODES_METADATA_DF["tone_start_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[1])

In [18]:
exploded_TRODES_METADATA_DF["tone_start_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[1])


In [19]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF.drop(columns=["tone_frames", "tone_timestamps"], errors="ignore")

- Create a new column for start and stop

In [20]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF[sorted(exploded_TRODES_METADATA_DF.columns, key=lambda s: s.split('_')[-1])].copy()

In [21]:
exploded_TRODES_METADATA_DF.to_pickle("./proc/{}_per_video_trial_labels.pkl".format(OUTPUT_PREFIX))

- Exporting to CSV

In [22]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: ' '.join(map(str, x)))

box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [23]:
exploded_TRODES_METADATA_DF.head()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240413_151204_comp_discriminate_subj_3-1_and...,1,66,[ 1 66],[ 1 66],20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,0,65243,[ 0 65243],[ 0 65243]
0,20240413_151204_comp_discriminate_subj_3-1_and...,1264,1464,,[1372 1373] [1376 1404] [1407 1410] [1411 1594],20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,1265655,1465660,,[1374459 1376056] [1377859 1406462] [1408460 1...
0,20240413_151204_comp_discriminate_subj_3-1_and...,3260,3459,,[3385 3386] [3388 3392] [3393 3394] [3397 3398...,20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,3265681,3465685,,[3390882 3391882] [3393685 3397482] [3398882 3...
0,20240413_151204_comp_discriminate_subj_3-1_and...,4458,4657,,[4471 4521] [4538 4541] [4541 4696],20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,4465698,4665701,,[4479098 4528699] [4546099 4548699] [4549500 4...
0,20240413_151204_comp_discriminate_subj_3-1_and...,5455,5656,,[5450 5457] [5460 5469] [5470 5471] [5505 5512...,20240413_151204_comp_discriminate_subj_3-1_and...,"[3.1, 3.3]",2604049,74833753,5465711,5665716,,[5459911 5465911] [5469909 5479111] [5479711 5...


In [24]:
exploded_TRODES_METADATA_DF.tail()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
25,20240417_150153_comp_discriminate_subj_3-3_and...,64047,64246,,,20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,64052556,64252558,,
25,20240417_150153_comp_discriminate_subj_3-3_and...,66242,66443,[66239 66250] [66253 66256] [66282 66284] [662...,,20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,66252586,66452588,[66249189 66260183] [66262789 66265584] [66292...,
25,20240417_150153_comp_discriminate_subj_3-3_and...,67340,67540,[67378 67424] [67426 67684],,20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,67352600,67552600,[67390598 67437199] [67437999 67696802],
25,20240417_150153_comp_discriminate_subj_3-3_and...,68938,69137,[68953 68955] [68964 68966] [68967 68970] [689...,,20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,68952620,69152620,[68967817 68970418] [68979818 68980620] [68982...,
25,20240417_150153_comp_discriminate_subj_3-3_and...,69935,70135,[69888 69974] [69976 70009] [70031 70386],,20240417_150153_comp_discriminate_subj_3-3_and...,"[3.3, 3.4]",880028,73742118,69952632,70152635,[69904832 69990833] [69993233 70025433] [70047...,


In [25]:
exploded_TRODES_METADATA_DF.to_csv("./proc/{}_per_video_trial_labels.csv".format(OUTPUT_PREFIX))